In [10]:
import gym
import torch
import torch.nn as nn


from models.td3 import TD3

In [11]:
class Actor(nn.Module):
    def __init__(self, n_sts: int, n_acts: int, dim: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(n_sts, dim)
        self.fc2 = nn.Linear(dim, dim)
        self.fc3 = nn.Linear(dim, n_acts)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = torch.tanh(self.fc3(x)) * 2
        
        return x

In [12]:
class Critic(nn.Module):
    def __init__(self, n_sts: int, n_acts: int, dim: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(n_sts + n_acts, dim)
        self.fc2 = nn.Linear(dim, 1)
        
        self.relu1 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.fc2(x)
        
        return x

In [13]:
lr = 1e-3
tau = 5e-3
eps = 0
gamma = 0.99

n_sts = 3
n_acts = 1

dim = 256

n_epis = 500
n_epochs = 200
n_rollout = 10

In [14]:
act = Actor(n_sts, n_acts, dim)
act_opt = torch.optim.Adam(act.parameters(), lr=lr)

cri1 = Critic(n_sts, n_acts, dim)
cri_opt1 = torch.optim.Adam(cri1.parameters(), lr=lr)

cri2 = Critic(n_sts, n_acts, dim)
cri_opt2 = torch.optim.Adam(cri2.parameters(), lr=lr)

In [15]:
env = gym.make('Pendulum-v1', render_mode='human')

In [16]:
model = TD3(env, n_acts, act, act_opt, cri1, cri_opt1, cri2, cri_opt2, eps=eps, tau=tau, act_noise=0.2, act_range = (-2, 2))

In [17]:
model.train(n_epis, n_epochs, n_rollout)

c:\Users\esder\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


step: 210, score: -81.19225413847353, n_buffer: 210
step: 410, score: -82.90910132102104, n_buffer: 410
step: 610, score: -84.88200366257588, n_buffer: 610
step: 810, score: -82.64932812686354, n_buffer: 810
step: 1010, score: -85.16061209315353, n_buffer: 1010
step: 1210, score: -82.36173603711453, n_buffer: 1210
step: 1410, score: -85.0647810127457, n_buffer: 1410
step: 1610, score: -82.76442127148397, n_buffer: 1610
step: 1810, score: -84.46114682047781, n_buffer: 1810
step: 2210, score: -84.2214457158984, n_buffer: 2210
step: 2410, score: -82.56493672049277, n_buffer: 2410
step: 2610, score: -84.93709444161627, n_buffer: 2610
step: 2810, score: -82.75847355417153, n_buffer: 2810
step: 3010, score: -84.66649241884872, n_buffer: 3010
step: 3210, score: -83.22741238205984, n_buffer: 3210
step: 3410, score: -83.79082387791475, n_buffer: 3410
step: 3610, score: -83.81790658378927, n_buffer: 3610
step: 3810, score: -83.42145503977133, n_buffer: 3810
step: 4210, score: -92.12086781877517,

KeyboardInterrupt: 

In [18]:
env.close()